In [2]:
import torch, numpy

In [7]:
import torch, math
from torch.autograd import Function
class MyMul(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return x * y

    @staticmethod
    def backward(ctx, grad_output):
        x,y = ctx.saved_tensors
        return grad_output * y,grad_output * x

class MyMax(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return torch.maximum(x,y)
    
    @staticmethod
    def backward(ctx, grad_output):
        x,y  = ctx.saved_tensors
        
        maskx = x > y
        masky = x < y
        maskequal = x == y 

        grad_x = torch.where(maskx, grad_output, torch.where(maskequal, grad_output * 0.5, torch.zeros_like(grad_output)))
        grad_y = torch.where(masky, grad_output, torch.where(maskequal, grad_output * 0.5, torch.zeros_like(grad_output)))
        
        return grad_x, grad_y 

class MyCos(Function):
    def forward(ctx, x):
        ctx.save_for_backward(x) 
        return torch.cos(x)
    def backward(ctx, gradient_output):
        x, = ctx.saved_tensors
        grad_input = gradient_output * -torch.sin(x)
        return grad_input

In [5]:
from torch.autograd import gradcheck

x = torch.randn(3, dtype=torch.double, requires_grad=True)
y = torch.randn(3, dtype=torch.double, requires_grad=True)

# Make sure inputs are a tuple of tensors
inputs = (x, y)

# Run gradcheck
print(gradcheck(MyMul.apply, inputs))
print(gradcheck(MyMax.apply, inputs))

True
True


In [ ]:
import torch 
import torch.nn as nn


class CosLinear(nn.Module):
    
    #Parameter initialization 
    def  __init__(self, num_inputs, num_outputs, hidden_size):
        super(CosLinear,self).__init__()
        self.linear1 = nn.Linear(num_inputs,hidden_size) # das darf ich nicht benutzen sonder muss von mir erstellt werden 

        self.linear2 = nn.Linear(hidden_size, num_outputs)
    #forward pass
    def forward(self, input):
        lin = self.linear1(input)
        output = nn.functional.relu(lin)
        pred = self.linear2(output)
        return pred 
    

In [ ]:
import torch, math 

class CosLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=False):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        if bias: 
            self.bias = torch.nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter("bias", None)
        

    def forward(self, input):
        weight_with_cos = MyCos.apply(self.weight)
        return nn.functional.linear(input, weight_with_cos, self.bias)
    
    
    


In [ ]:
import torch, math, torchvision
import torch.nn as nn
import torch.nn.functional as F 

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1,6,5)


    def forward(self,input):
        c1 = F.relu(self.con)